In [32]:
#import libraries
import bs4 as bs
import urllib.request
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import re
import pandas as pd
from urllib.request import urlopen


#Given a URL, find the homepage of the website (sometimes you may be given a URL of an inner page)
def findhomepage(website):
    count=0
    homepage=""
    for i in website:
        if i=="/":
            count+=1
        if count!=3:    #the homepage will be present in the url before the 3rd '/' . eg: https:/(1/2www.exeterpremedia.com/3
            homepage+=i
        elif count==3:
            break
    return(homepage)


#Find the links in the home page (and remove duplicates)  &&  Store the links
def getandstorelinks(website):
    homepage=findhomepage(website)
    source = urllib.request.urlopen(website).read()
    soup = bs.BeautifulSoup(source,'lxml')
    links=[]
    for url in soup.find_all('a'):              #finding all anchor tags
        links.append(url.get('href'))        # getting all tags that have href
    d={}                                                #dictionary is created to remove duplicate links
    unique_links=[]
    for i in links:
        if i not in d:
            d[i]=1
            if i and i[0]=="h":                       #to remove unnecessary anchor tags and get only appropriate links i.e links starting with "https"
                unique_links.append(i)
    return(unique_links)


#For each link, get the web page, extract text and store it
def getwebpageandstoretext(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    textineachlink=[]
    for eachlink in unique_links:
        if homepage in eachlink:
            source = urllib.request.urlopen(eachlink).read()
            soup = bs.BeautifulSoup(source,'lxml')
            textineachlink.append(soup.text)                                 #using soup.text to extract text from the webpage
    return textineachlink


#Parse the text (remove stop words, punctuation) and generate a list of uni-grams, and bi-grams from the text. We will refer to these as key terms.
def removepunctuationsandstopwords(website):  
    source = urllib.request.urlopen(website).read()
    soup = bs.BeautifulSoup(source,'lxml')
    sentence=soup.text
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(sentence) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)                                #remove stop words
    string=""
    for i in filtered_sentence:
        string=string+" "+i
    string = re.sub(r'[^\w\s]', '', string)                             #remove punctuations
    return string



#Function to generate unigrams
def uni_grams(sentence):
    dic={}
    unigram=[]
    sentence=sentence.split()
    return sentence

#Function to generate bigrams
def bi_grams(sentence):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    l = re.split(r' [\.\?!][\'"\)\]] *', sentence)
  #print(l)
  #return l
    if len(l[len(l)-1])==0:
        l.pop()
  #return l 
    ans=[]
    for i in l:
        kish=list(i.split())
        for j in range(1,len(kish)):
            str1=kish[j-1]
            str2=kish[j]
            tmp = ""
            for k in str1:
                if k not in punc and k!=' ' :
                    tmp += k
            str1=tmp.lower()
            tmp = ""
            for k in str2:
                if k not in punc and k!=' ' :
                    tmp += k
            str2=tmp.lower()
            ans.append(str1+" "+str2)
    return ans

#function to generate trigrams
def tri_grams(sentence):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    l = re.split(r' [\.\?!][\'"\)\]] *', sentence)
  #print(l)
  #return l
    if len(l[len(l)-1])==0:
        l.pop()
  #return l 
    ans=[]
    for i in l:
        kish=list(i.split())
        for j in range(2,len(kish)):
            str1=kish[j-2]
            str2=kish[j-1]
            str3=kish[j]
            tmp = ""
            for k in str1:
                if k not in punc and k!=' ' :
                    tmp += k
            str1=tmp.lower()
            tmp = ""
            for k in str2:
                if k not in punc and k!=' ' :
                    tmp += k
            str2=tmp.lower()
            tmp = ""
            for k in str3:
                if k not in punc and k!=' ' :
                    tmp += k
            str3=tmp.lower()
            ans.append(str1+" "+str2+" "+str3)
    return ans



# The number of top level pages (a top level page is  one that has a link on the home page)
def numberoftoplevelpages(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    return(len(unique_links))


#Page titles of all the top level pages 
def topleveltitle(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    titles=[]
    for i in unique_links:
        if homepage in i:
            source = urllib.request.urlopen(i).read()
            soup = bs.BeautifulSoup(source,'lxml')
            titles.append(soup.title)
    return titles


#Meta tags from all top level pages
def meta_tags(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    metatags=[]
    for i in unique_links:
        if i and homepage in i:
            source = urllib.request.urlopen(i).read()
            soup = bs.BeautifulSoup(source,'lxml')
            metatags.append(soup.find('meta'))
    return metatags

#All internal links (links that point to some page in the same website)
def internal_links(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    internallinks=[]
    for i in unique_links:
        if  i and homepage in i:             #checking whether the website name is in the link
            internallinks.append(i)
    return internallinks

#All external links (links that point to pages outside the site). 
def external_links(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    externallinks=[]
    for i in unique_links:
        if  i and homepage not  in i:                 #checking whether the website name is not in the link
            externallinks.append(i)
    return externallinks    

#A frequency table of the top 20 key terms (unigrams and bigrams) in the descending order of frequency and write out a CSV file.
def findtop20(ngram):
    d={}
    top20=[]
    for i in ngram:
        if i not in d:
            d[i]=1
        else:
            d[i]+=1
    sorteddict = sorted(d, key=d.get, reverse=True)             #sorting the dictionary based on keys in descending order
    for i in range(20):
        top20.append(sorteddict[i])
    return top20

#Find the size of the web page
def findsizeofwebpage(website):
    r = urlopen(website)
    return len(r.read())

print('\033[1m' +"Enter the website URL")                          #'\033[1m' is used to print the text bold in python
website_name=input()
homepage_name=findhomepage(website_name)

linksinhomepage=[]
linksinhomepage=getandstorelinks(website_name)

textineachpage=getwebpageandstoretext(website_name)

clear_sentence=removepunctuationsandstopwords(website_name)
unigrams=uni_grams(clear_sentence)
bigrams=bi_grams(clear_sentence)
trigrams=tri_grams(clear_sentence)

number_of_top_level_pages=numberoftoplevelpages(website_name)

titles_list=topleveltitle(website_name)

meta_tags_list=meta_tags(website_name)

internal_links_list=internal_links(website_name)

external_links_list=external_links(website_name)

top20unigrams=findtop20(unigrams)
top20bigrams=findtop20(bigrams)
top20trigrams=findtop20(trigrams)

sizeofwebpage=findsizeofwebpage(website_name)



Enter the website URL
https://www.exeterpremedia.com/


In [36]:
#printing the output in a csv file

dictionary={"Homepage": homepage_name, 
            "Links in homepage":linksinhomepage,
            "Text in each link" :textineachpage,
            "Unigrams" : unigrams,
            "Bigrams" : bigrams,
            "Trigrams" : trigrams,
            "Number of top level pages":number_of_top_level_pages,
            "Page titles" :titles_list,
            "Meta tags" : meta_tags_list,
            "All internal links" : internal_links_list,
            "All external links" : external_links_list,
           "Size of web page" : sizeofwebpage}

df=pd.DataFrame.from_dict(dictionary, orient='index')

df.to_csv("featuresandoutput.csv")

#6. A frequency table of the top 20 key terms (unigrams and bigrams) in the descending order of frequency and write out a CSV file.
top20dict={"TOP 20 UNIGRAMS" : top20unigrams, "TOP 20 BIGRAMS" : top20bigrams , "TOP 20 TRIGRAMS" : top20trigrams}
df1=pd.DataFrame(top20dict) 

df1.to_csv("top20ngrams.csv")

In [35]:
#printing the output in a text file

print("INPUT SITE",file=open("output.txt", "a"))
print(website_name,file=open("output.txt", "a"))

print("\nHOME PAGE NAME",file=open("output.txt", "a"))
print(homepage_name,file=open("output.txt", "a"))

print("\nLINKS IN HOME PAGE",file=open("output.txt", "a"))
print(*linksinhomepage,sep="\n",file=open("output.txt", "a"))

print("\nNUMBER OF TOP LEVEL PAGES", file=open("output.txt", "a"))
print(number_of_top_level_pages, file=open("output.txt", "a"))

print("\nPAGE TITLES OF ALL TOP LEVEL PAGES", file=open("output.txt", "a"))
print(*titles_list,sep="\n", file=open("output.txt", "a"))

print("\nMETA TAGS OF TOP LEVEL PAGES", file=open("output.txt", "a"))
print(*meta_tags_list,sep="\n", file=open("output.txt", "a"))

print("\nALL INTERNAL LINKS", file=open("output.txt", "a"))
print(*internal_links_list,sep="\n", file=open("output.txt", "a"))

print("\nALL EXTERNAL LINKS", file=open("output.txt", "a"))
print(*external_links_list,sep="\n", file=open("output.txt", "a"))

print("\nTOP 20 UNIGRAMS", file=open("output.txt", "a"))
print(*top20unigrams,sep="\n", file=open("output.txt", "a"))

print("\nTOP 20 BIGRAMS", file=open("output.txt", "a"))
print(*top20bigrams,sep="\n", file=open("output.txt", "a"))

print("\nTOP 20 TRIGRAMS", file=open("output.txt", "a"))
print(*top20trigrams,sep="\n", file=open("output.txt", "a"))

print("\nSIZE OF WEB PAGE", file=open("output.txt", "a"))
print(sizeofwebpage, file=open("output.txt", "a"))


In [34]:
#printing the ouput in jupyter notebook
print('\033[1m' +"INPUT SITE")
print(website_name)

print('\033[1m' +"\nHOME PAGE NAME")
print(homepage_name)

print('\033[1m' +"\nLINKS IN HOME PAGE")
print(*linksinhomepage,sep="\n")

print('\033[1m' +"\nTEXT IN EACH PAGE")
print(*textineachpage,sep="\n")

print('\033[1m' +"\nTEXT AFTER REMOVING PUNCTUATIONS AND STOP WORDS")
print(clear_sentence)

print('\033[1m' +"\nUNIGRAMS")
print(*unigrams,sep="\n")

print('\033[1m' +"\nBIGRAMS")
print(*bigrams,sep="\n")

print('\033[1m' +"\nTRIGRAMS")
print(*trigrams,sep="\n")

print('\033[1m' +"\nNUMBER OF TOP LEVEL PAGES")
print(number_of_top_level_pages)

print('\033[1m' +"\nPAGE TITLES OF ALL TOP LEVEL PAGES")
print(*titles_list,sep="\n")

print('\033[1m' +"\nMETA TAGS OF TOP LEVEL PAGES")
print(*meta_tags_list,sep="\n")

print('\033[1m' +"\nALL INTERNAL LINKS")
print(*internal_links_list,sep="\n")

print('\033[1m' +"\nALL EXTERNAL LINKS")
print(*external_links_list,sep="\n")

print('\033[1m' +"\nTOP 20 UNIGRAMS")
print(*top20unigrams,sep="\n")

print('\033[1m' +"\nTOP 20 BIGRAMS")
print(*top20bigrams,sep="\n")

print('\033[1m' +"\nTOP 20 TRIGRAMS")
print(*top20trigrams,sep="\n")

print('\033[1m' +"\nSIZE OF WEB PAGE")
print(sizeofwebpage)

INPUT SITE
https://www.exeterpremedia.com/

HOME PAGE NAME
https://www.exeterpremedia.com

LINKS IN HOME PAGE
https://www.kriyadocs.com/
https://www.exeterpremedia.com/services/
https://www.exeterpremedia.com/services/#Editorial_Services
https://www.exeterpremedia.com/services/#Data_Services
https://www.exeterpremedia.com/services/#Artwork
https://www.exeterpremedia.com/services/#Project_Management
https://www.exeterpremedia.com/blog/
https://www.exeterpremedia.com/about-us/
https://www.exeterpremedia.com/events/
https://www.exeterpremedia.com/news/
https://www.exeterpremedia.com/contact-us/
https://careers.exeterpremedia.com/jobs/Careers
https://www.exeterpremedia.com/
https://www.exeterpremedia.com/wp-content/uploads/2019/06/atma-global-inc.webp
https://www.exeterpremedia.com/wp-content/uploads/2019/06/bmj-group.webp
https://www.exeterpremedia.com/wp-content/uploads/2019/06/bolinda-publishing-llc.webp
https://www.exeterpremedia.com/wp-content/uploads/2019/06/brill.webp
https://www.ex

rotateX
180deg
rotateY
180deg
75
webkittransform
perspective
100px
rotateX
0
rotateY
180deg
transform
perspective
100px
rotateX
0
rotateY
180deg
100
webkittransform
perspective
100px
rotateX
0
rotateY
0
transform
perspective
100px
rotateX
0
rotateY
0
squarespin
display
inlineblock
squarespin
div
webkitanimationfillmode
animationfillmode
width50px
height50px
webkitanimation
squarespin
3s
0s
cubicbezier
09
57
49
9
infinite
animation
squarespin
3s
0s
cubicbezier
09
57
49
9
infinite
cubetransition
position
relative
webkittransform
translate
25px
25px
mstransform
translate
25px
25px
transform
translate
25px
25px
display
inlineblock
cubetransition
div
webkitanimationfillmode
animationfillmode
width15px
height15px
position
absolute
top
5px
left
5px
webkitanimation
cubetransition
16s
0s
infinite
easeinout
animation
cubetransition
16s
0s
infinite
easeinout
cubetransition
div
lastchild
webkitanimationdelay
8s
animationdelay
8s
webkitkeyframes
cubetransition
25
webkittransform
translateX
50px
sca

webkitboxshadow0
0
px
rgba
000
mozboxshadow0
0
px
rgba
000
boxshadow0
0
px
rgba
000
mknewstab
mktabstabs
isactive
mkfancytitlepatternstyle
span
mkfancytitlepatternstylecolorgradient
span
pagebgcolor
backgroundcolor
fff
pagetitle
fontsize20px
color
4d4d4d
texttransform
uppercase
fontweight400
letterspacing2px
pagesubtitle
fontsize14px
lineheight100
color
a3a3a3
fontsize14px
texttransform
none
mkheader
borderbottom1px
solid
ededed
headerstyle1
mkheaderpaddingwrapper
headerstyle2
mkheaderpaddingwrapper
headerstyle3
mkheaderpaddingwrapper
paddingtop91px
body
fontfamily
Lato
WebFontConfig
timeout2000
mk_typekit_idlength
0
WebFontConfigtypekit
id
mk_typekit_id
mk_google_fontslength
0
WebFontConfiggoogle
families
mk_google_fonts
mk_google_fontslength
0mk_typekit_idlength
0
navigatoruserAgentindexOf
Speed
Insights
1
WebFontload
WebFontConfig
CDATA
var
LS_Meta
v
676
CDATA
var
BP_DTheme
accepted
Accepted
close
Close
comments
comments
leave_group_confirm
Are
sure
want
leave
group
mark_as_fav
Favo

3619
3634
3585
3634
3619
TH74
Samut
Sakhon
3626
3617
3640
3607
3619
3626
3634
3588
3619
TH75
Samut
Songkhram
3626
3617
3640
3607
3619
3626
3591
3588
3619
3634
3617
TH19
Saraburi
3626
3619
3632
3610
3640
3619
3637
TH91
Satun
3626
3605
3641
3621
TH17
Sing
Buri
3626
3636
3591
3627
3660
3610
3640
3619
3637
TH33
Sisaket
3624
3619
3637
3626
3632
3648
3585
3625
TH90
Songkhla
3626
3591
3586
3621
3634
TH64
Sukhothai
3626
3640
3650
3586
3607
3633
3618
TH72
Suphan
Buri
3626
3640
3614
3619
3619
3603
3610
3640
3619
3637
TH84
Surat
Thani
3626
3640
3619
3634
3625
3598
3619
3660
3608
3634
3609
3637
TH32
Surin
3626
3640
3619
3636
3609
3607
3619
3660
TH63
Tak
3605
3634
3585
TH92
Trang
3605
3619
3633
3591
TH23
Trat
3605
3619
3634
3604
TH34
Ubon
Ratchathani
3629
3640
3610
3621
3619
3634
3594
3608
3634
3609
3637
TH41
Udon
Thani
3629
3640
3604
3619
3608
3634
3609
3637
TH61
Uthai
Thani
3629
3640
3607
3633
3618
3608
3634
3609
3637
TH53
Uttaradit
3629
3640
3605
3619
3604
3636
3605
3606
3660
TH95
Yala
3618
3632

Lakshmipur
LAL
Lalmonirhat
MAD
Madaripur
MAG
Magura
MAN
Manikganj
MEH
Meherpur
MOU
Moulvibazar
MUN
Munshiganj
MYM
Mymensingh
NAO
Naogaon
NAR
Narail
NARG
Narayanganj
NARD
Narsingdi
NAT
Natore
NAW
Nawabganj
NET
Netrakona
NIL
Nilphamari
NOA
Noakhali
PAB
Pabna
PAN
Panchagarh
PAT
Patuakhali
PIR
Pirojpur
RAJB
Rajbari
RAJ
Rajshahi
RAN
Rangamati
RANP
Rangpur
SAT
Satkhira
SHA
Shariatpur
SHE
Sherpur
SIR
Sirajganj
SUN
Sunamganj
SYL
Sylhet
TAN
Tangail
THA
Thakurgaon
IR
KHZ
Khuzestan
u062eu0648u0632u0633u062au0627u0646
THR
Tehran
u062au0647u0631u0627u0646
ILM
Ilaam
u0627u06ccu0644u0627u0645
BHR
Bushehr
u0628u0648u0634u0647u0631
ADL
Ardabil
u0627u0631u062fu0628u06ccu0644
ESF
Isfahan
u0627u0635u0641u0647u0627u0646
YZD
Yazd
u06ccu0632u062f
KRH
Kermanshah
u06a9u0631u0645u0627u0646u0634u0627u0647
KRN
Kerman
u06a9u0631u0645u0627u0646
HDN
Hamadan
u0647u0645u062fu0627u0646
GZN
Ghazvin
u0642u0632u0648u06ccu0646
ZJN
Zanjan
u0632u0646u062cu0627u0646
LRS
Luristan
u0644u0631u0633u062au0627u0646
ABZ
Alborz
u0627

...

laga ml melilla
ml melilla mu
melilla mu murcia
mu murcia na
murcia na navarra
na navarra or
navarra or ourense
or ourense p
ourense p palencia
p palencia po
palencia po pontevedra
po pontevedra sa
pontevedra sa salamanca
sa salamanca tf
salamanca tf santa
tf santa cruz
santa cruz de
cruz de tenerife
de tenerife sg
tenerife sg segovia
sg segovia se
segovia se sevilla
se sevilla so
sevilla so soria
so soria t
soria t tarragona
t tarragona te
tarragona te teruel
te teruel to
teruel to toledo
to toledo v
toledo v valencia
v valencia va
valencia va valladolid
va valladolid bi
valladolid bi bizkaia
bi bizkaia za
bizkaia za zamora
za zamora z
zamora z zaragoza
z zaragoza ca
zaragoza ca ab
ca ab alberta
ab alberta bc
alberta bc british
bc british columbia
british columbia mb
columbia mb manitoba
mb manitoba nb
manitoba nb new
nb new brunswick
new brunswick nl
brunswick nl newfoundland
nl newfoundland labrador
newfoundland labrador nt
labrador nt northwest
nt northwest territories
northwest te

tr76 i 287
i 287 305
287 305 r
305 r tr77
r tr77 yalova
tr77 yalova tr78
yalova tr78 karab
tr78 karab 252
karab 252 k
252 k tr79
k tr79 kilis
tr79 kilis tr80
kilis tr80 osmaniye
tr80 osmaniye tr81
osmaniye tr81 d
tr81 d 252
d 252 zce
252 zce hk
zce hk hong
hk hong kong
hong kong hong
kong hong kong
hong kong island
kong island kowloon
island kowloon kowloon
kowloon kowloon new
kowloon new territories
new territories new
territories new territories
new territories var
territories var ajaxurlhttps
var ajaxurlhttps wwwexeterpremediacomwpadminadminajaxphp
ajaxurlhttps wwwexeterpremediacomwpadminadminajaxphp var
wwwexeterpremediacomwpadminadminajaxphp var msgrabbingcurosrhttps
var msgrabbingcurosrhttps wwwexeterpremediacomwpcontentpluginsmastersliderpublicassetscsscommongrabbingcur
msgrabbingcurosrhttps wwwexeterpremediacomwpcontentpluginsmastersliderpublicassetscsscommongrabbingcur msgrabcurosrhttps
wwwexeterpremediacomwpcontentpluginsmastersliderpublicassetscsscommongrabbingcur msgrabcuro

top158px important loop11
important loop11 194
loop11 194 top158px
194 top158px important
top158px important mkblognewspaperitemonecolumn
important mkblognewspaperitemonecolumn height170px
mkblognewspaperitemonecolumn height170px mktwittershortcode
height170px mktwittershortcode color
mktwittershortcode color 1da1f2
color 1da1f2 important
1da1f2 important top6px
important top6px position
top6px position relative
position relative customlinkedinpage
relative customlinkedinpage wpbwrapper
customlinkedinpage wpbwrapper overflowy
wpbwrapper overflowy scroll
overflowy scroll height507px
scroll height507px padding8px
height507px padding8px yourname
padding8px yourname width100
yourname width100 position
width100 position relative
position relative top10px
relative top10px borderradius5px
top10px borderradius5px border1px
borderradius5px border1px solid
border1px solid f3f3f3
solid f3f3f3 height50px
f3f3f3 height50px youremail
height50px youremail width100
youremail width100 position
width100

23aaa 3b 0a
3b 0a 20
0a 20 20
20 20 20
20 20 20background
20 20background 3argba
20background 3argba 28125
3argba 28125 2c125
28125 2c125 2c125
2c125 2c125 2c05
2c125 2c05 29
2c05 29 3b
29 3b 0a
3b 0a 09cursor
0a 09cursor 3a
09cursor 3a 20pointer
3a 20pointer 3b
20pointer 3b 0a
3b 0a 09boxsizing
0a 09boxsizing 3acontentbox
09boxsizing 3acontentbox 3b
3acontentbox 3b 0a
3b 0a 7d
0a 7d 0acustom
7d 0acustom 20tpbullet
0acustom 20tpbullet 3ahover
20tpbullet 3ahover 2c
3ahover 2c 0acustom
2c 0acustom 20tpbulletselected
0acustom 20tpbulletselected 20
20tpbulletselected 20 7b
20 7b 0a
7b 0a 09background
0a 09background 3argb
09background 3argb 28125
3argb 28125 2c125
28125 2c125 2c125
2c125 2c125 29
2c125 29 3b
29 3b 0a
3b 0a 7d
0a 7d 0acustom
7d 0acustom 20tpbulletimage
0acustom 20tpbulletimage 20
20tpbulletimage 20 7b
20 7b 0a
7b 0a 7d
0a 7d 0acustom
7d 0acustom 20tpbullettitle
0acustom 20tpbullettitle 20
20tpbullettitle 20 7b
20 7b 0a
7b 0a 7d
0a 7d 0a
7d 0a 0a
0a 0a var
0a var htmldivdocu

httpbuildqueryselectively httpbuildqueryselectivelyfields delayredirect
httpbuildqueryselectivelyfields delayredirect 0
delayredirect 0 aftersentscript
0 aftersentscript thankyoupageurl
aftersentscript thankyoupageurl 2146
thankyoupageurl 2146 pageid
2146 pageid 0
pageid 0 externalurl
0 externalurl useexternalurl
externalurl useexternalurl openinnewtab
useexternalurl openinnewtab httpbuildquery
openinnewtab httpbuildquery httpbuildqueryselectively
httpbuildquery httpbuildqueryselectively httpbuildqueryselectivelyfields
httpbuildqueryselectively httpbuildqueryselectivelyfields delayredirect
httpbuildqueryselectivelyfields delayredirect 0
delayredirect 0 aftersentscript
0 aftersentscript thankyoupageurl
aftersentscript thankyoupageurl 2144
thankyoupageurl 2144 pageid
2144 pageid 0
pageid 0 externalurl
0 externalurl useexternalurl
externalurl useexternalurl openinnewtab
useexternalurl openinnewtab httpbuildquery
openinnewtab httpbuildquery httpbuildqueryselectively
httpbuildquery httpbuil